# Analysis of App Store and Google Play apps

In this project, we are pretending to be working as data analyst for company that builds mobile apps. 

The task would be analyze apps from platforms mentioned above. The main purpose of such task will be to find apps that atracts more users to decide what kind of apps we should focus on. Our field of interest consider just English speaking customers and we also produce free apps only, profiting from adds.

In the first step, we would create app that fits the most popular category. If the app has good response in the future, we are going to deploy iOS version on iOS App Store. So, we need to find app patterns, that would most likely be successful on both platforms.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
pd.options.mode.chained_assignment = None 
import seaborn as sns

## Getting the data 

In fact, there is around 4 milion apps in the market. Such large amount of data would need 
a lot of time resources, so we could try to find if there are any datasets available online.

Fortunately, Kaggle.com provides a great use for our case - two promising datasets containing app data information could be found on the site:

[Android Google Play Store](https://www.kaggle.com/lava18/google-play-store-apps)

[Apple iOS App Store](https://www.kaggle.com/ramamet4/app-store-apple-data-set-10k-apps)


### Importing 

In [2]:
ggl = pd.read_csv('googleplaystore.csv')
ios = pd.read_csv('AppleStore.csv') 

print('Google Play dataset shape (rows, columns): ', ggl.shape,end='\n\n')
print('iOS App Store dataset shape (rows, columns): ',ios.shape)

Google Play dataset shape (rows, columns):  (10841, 13)

iOS App Store dataset shape (rows, columns):  (7197, 16)


In [3]:
ggl.head()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up


In [4]:
ios.head()

,id,track_name,size_bytes,currency,price,rating_count_tot,rating_count_ver,user_rating,user_rating_ver,ver,cont_rating,prime_genre,sup_devices.num,ipadSc_urls.num,lang.num,vpp_lic
0,284882215,Facebook,389879808,USD,0.0,2974676,212,3.5,3.5,95.0,4+,Social Networking,37,1,29,1
1,389801252,Instagram,113954816,USD,0.0,2161558,1289,4.5,4.0,10.23,12+,Photo & Video,37,0,29,1
2,529479190,Clash of Clans,116476928,USD,0.0,2130805,579,4.5,4.5,9.24.12,9+,Games,38,5,18,1
3,420009108,Temple Run,65921024,USD,0.0,1724546,3842,4.5,4.0,1.6.2,9+,Games,40,5,1,1
4,284035177,Pandora - Music & Radio,130242560,USD,0.0,1126879,3594,4.0,4.5,8.4.1,12+,Music,37,4,1,1


#### Google Play Store columns:

Dataset scrapped: July 2018

- **"App"**: Application name

- **"Category"**: Category the app belongs to

- **"Rating"**: Overall user rating of the app (as when scraped)

- **"Reviews"**: Number of user reviews for the app (as when scraped)

- **"Size"**: Size of the app (as when scraped)

- **"Installs"**: Number of user downloads/installs for the app (as when scraped)

- **"Type"**: Paid or Free 

- **"Price"**: Price of the app (as when scraped)

- **"Content Rating"**: Age group the app is targeted at - Children / Mature 21+ / Adult

- **"Genres"**: An app can belong to multiple genres (apart from its main category). For eg, a musical family game will belong to

- **"Last Updated"**: Date when the app was last updated on Play Store (as when scraped)

- **"Current Ver"**: Current version of the app available on Play Store (as when scraped)

- **"Android Ver"**: Min required Android version (as when scraped)

#### iOS App Store columns:

Dataset scrapped: July 2017

- **"id"** : App ID

- **"track_name"**: App Name

- **"size_bytes"**: Size (in Bytes)

- **"currency"**: Currency Type

- **"price"**: Price amount

- **"ratingcounttot"**: User Rating counts (for all version)

- **"ratingcountver"**: User Rating counts (for current version)

- **"user_rating"**  : Average User Rating value (for all version)

- **"userratingver"**: Average User Rating value (for current version)

- **"ver"** : Latest version code

- **"cont_rating"**: Content Rating

- **"prime_genre"**: Primary Genre

- **"sup_devices.num"**: Number of supporting devices

- **"ipadSc_urls.num"**: Number of screenshots showed for display

- **"lang.num"**: Number of supported languages

- **"vpp_lic"**: Vpp Device Based Licensing Enabled

As we can see from columns descriptions, the main columns of our interest are probably the name of the app, genre, price and correspodning ratings.

## Exploration and cleaning

### **1) Google Play**

**Missing values:**

In [5]:
ggl.isna().sum()

App                  0
Category             0
Rating            1474
Reviews              0
Size                 0
Installs             0
Type                 1
Price                0
Content Rating       1
Genres               0
Last Updated         0
Current Ver          8
Android Ver          3
dtype: int64

Ratings has the most total values of all columns. We will leave it as it is since there still could be useful information in these rows. Let's also take a quick look on missing values from "Type" column - there is just one case where this feature is missing:

In [6]:
ggl[ggl['Type'].isna()]

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
9148,Command & Conquer: Rivals,FAMILY,NaN,0,Varies with device,0,NaN,0,Everyone 10+,Strategy,"June 28, 2018",Varies with device,Varies with device


"Content Rating" column has also one missing value. We can notice that row has more mistakes - "Rating" should be in range 1-5, and "Category" is always numerical variable in other cases. Considering these flaws, we are goind to delete this entry.

In [7]:
ggl[ggl['Content Rating'].isna()]

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
10472,Life Made WI-Fi Touchscreen Photo Frame,1.9,19.0,3.0M,"1,000+",Free,0,Everyone,NaN,"February 11, 2018",1.0.19,4.0 and up,NaN


In [8]:
ggl.drop(index=10472, inplace=True)

Last two columns also contain missing values, but they don't seem crucial for the analysis task, so we will ommit them for now.

**Duplicates:**

Good idea in this case is check for duplicates. We can see there is quite a few of them. Let's also have a look on an example:

In [9]:
ggl.duplicated().sum()

483

In [10]:
ggl[ggl['App'] == 'WhatsApp Messenger']

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
336,WhatsApp Messenger,COMMUNICATION,4.4,69119316,Varies with device,"1,000,000,000+",Free,0,Everyone,Communication,"August 3, 2018",Varies with device,Varies with device
381,WhatsApp Messenger,COMMUNICATION,4.4,69119316,Varies with device,"1,000,000,000+",Free,0,Everyone,Communication,"August 3, 2018",Varies with device,Varies with device
3904,WhatsApp Messenger,COMMUNICATION,4.4,69109672,Varies with device,"1,000,000,000+",Free,0,Everyone,Communication,"August 3, 2018",Varies with device,Varies with device


For example WhatsApp Messenger app has three entries with almost identical stats. We can spot the difference in Reviews column - the number of given reviews is not consistent. It can be concluded that the data entry with the most reviews is the most actual. Due to this, we want to keep just this entry and delete other duplicates. The similar approach is used for all duplicated apps in the dataset.

In [11]:
ggl_no_dups = ggl.groupby('App').max().reset_index()

In [12]:
ggl_no_dups[ggl_no_dups['App'] == 'WhatsApp Messenger']

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
9041,WhatsApp Messenger,COMMUNICATION,4.4,69119316,Varies with device,"1,000,000,000+",Free,0,Everyone,Communication,"August 3, 2018",Varies with device,Varies with device


In [13]:
ggl_no_dups.head()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,"""i DT"" Fútbol. Todos Somos Técnicos.",SPORTS,NaN,27,3.6M,500+,Free,0,Everyone,Sports,"October 7, 2017",0.22,4.1 and up
1,+Download 4 Instagram Twitter,SOCIAL,4.5,40467,22M,"1,000,000+",Free,0,Everyone,Social,"August 2, 2018",5.03,4.1 and up
2,- Free Comics - Comic Apps,COMICS,3.5,115,9.1M,"10,000+",Free,0,Mature 17+,Comics,"July 13, 2018",5.0.12,5.0 and up
3,.R,TOOLS,4.5,259,203k,"10,000+",Free,0,Everyone,Tools,"September 16, 2014",1.1.06,1.5 and up
4,/u/app,COMMUNICATION,4.7,573,53M,"10,000+",Free,0,Mature 17+,Communication,"July 3, 2018",4.2.4,4.1 and up


This procedure dropped about 1000 enties.

**Filtering just english-localized apps:**

We want to analyze just english - localized market. To achieve this, it is convenient to use ANSII coding of characters. Many apps in the dataset contain emojis - this could be solved by installing emoji library. Custom function below checks whether the string in "App" column contains only characters ASCII (having result of ord() function less 127 or less) or if the string is unicode corresponding for certain emoji. If both criterions are met, the function outputs True.

In following step, we pass english() function into apply method used on Series, where it't implemented into anonymous lambda function. The result is saved into new column, "English".

If we continue to explore dataset afterwards, we can notice that there are some apps misclassified - they most likely contain a symbol that lies out of ASCII range or unicodes for emojis. However, it seems that the majority of data was filtered properly, so we keep the labeling english() function as it is.

In [14]:
#%pip install emoji
from emoji import UNICODE_EMOJI

def english(s):
    
    return all(((ord(c) < 128) or (c in UNICODE_EMOJI)) for c in s) 

In [15]:
ggl_no_dups['English'] = ggl_no_dups['App'].apply(lambda app_name: english(app_name))

In [16]:
ggl2 = ggl_no_dups[ggl_no_dups['English'] == True]
ggl2.head()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver,English
1,+Download 4 Instagram Twitter,SOCIAL,4.5,40467,22M,"1,000,000+",Free,0,Everyone,Social,"August 2, 2018",5.03,4.1 and up,True
2,- Free Comics - Comic Apps,COMICS,3.5,115,9.1M,"10,000+",Free,0,Mature 17+,Comics,"July 13, 2018",5.0.12,5.0 and up,True
3,.R,TOOLS,4.5,259,203k,"10,000+",Free,0,Everyone,Tools,"September 16, 2014",1.1.06,1.5 and up,True
4,/u/app,COMMUNICATION,4.7,573,53M,"10,000+",Free,0,Mature 17+,Communication,"July 3, 2018",4.2.4,4.1 and up,True
5,058.ba,NEWS_AND_MAGAZINES,4.4,27,14M,100+,Free,0,Everyone,News & Magazines,"July 6, 2018",1.0,4.2 and up,True


**Free apps only:**

We can see that all the apps having "Type" as Free has correctly assigned "Price" value to zero. Though it doesn't matter which one of these features we use. I will use basic pandas boolean filtering by "Type" to get free apps only.

In [17]:
ggl2[(ggl2['Type'] =='Free') & (ggl2['Price'] != '0')]

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver,English


In [18]:
free_ggl2 = ggl2[ggl2['Type'] == 'Free']
free_ggl2.head()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver,English
1,+Download 4 Instagram Twitter,SOCIAL,4.5,40467,22M,"1,000,000+",Free,0,Everyone,Social,"August 2, 2018",5.03,4.1 and up,True
2,- Free Comics - Comic Apps,COMICS,3.5,115,9.1M,"10,000+",Free,0,Mature 17+,Comics,"July 13, 2018",5.0.12,5.0 and up,True
3,.R,TOOLS,4.5,259,203k,"10,000+",Free,0,Everyone,Tools,"September 16, 2014",1.1.06,1.5 and up,True
4,/u/app,COMMUNICATION,4.7,573,53M,"10,000+",Free,0,Mature 17+,Communication,"July 3, 2018",4.2.4,4.1 and up,True
5,058.ba,NEWS_AND_MAGAZINES,4.4,27,14M,100+,Free,0,Everyone,News & Magazines,"July 6, 2018",1.0,4.2 and up,True


### **2) iOS App Store**

Exploration of iOS dataset goes in the same manner as above.

**Missing values:**

As we can see, the iOS App Store dataset is already "clean" in terms of missing values.

In [19]:
ios.isna().sum()

id                  0
track_name          0
size_bytes          0
currency            0
price               0
rating_count_tot    0
rating_count_ver    0
user_rating         0
user_rating_ver     0
ver                 0
cont_rating         0
prime_genre         0
sup_devices.num     0
ipadSc_urls.num     0
lang.num            0
vpp_lic             0
dtype: int64

**Duplicates:**

Neither missing values nor duplicates are present in the dataset. Someone did a good job scrapping the data.

In [20]:
ios.duplicated().sum()

0

**Filtering just english-localized apps:**

In [21]:
ios['English'] = ios['track_name'].apply(lambda app_name: english(app_name))
ios2 = ios[ios['English'] == True]

**Free apps only:**

In [22]:
free_ios2 = ios2[ios2['price'] == 0]
free_ios2.head()

,id,track_name,size_bytes,currency,price,rating_count_tot,rating_count_ver,user_rating,user_rating_ver,ver,cont_rating,prime_genre,sup_devices.num,ipadSc_urls.num,lang.num,vpp_lic,English
0,284882215,Facebook,389879808,USD,0.0,2974676,212,3.5,3.5,95.0,4+,Social Networking,37,1,29,1,True
1,389801252,Instagram,113954816,USD,0.0,2161558,1289,4.5,4.0,10.23,12+,Photo & Video,37,0,29,1,True
2,529479190,Clash of Clans,116476928,USD,0.0,2130805,579,4.5,4.5,9.24.12,9+,Games,38,5,18,1,True
3,420009108,Temple Run,65921024,USD,0.0,1724546,3842,4.5,4.0,1.6.2,9+,Games,40,5,1,1,True
4,284035177,Pandora - Music & Radio,130242560,USD,0.0,1126879,3594,4.0,4.5,8.4.1,12+,Music,37,4,1,1,True


## Most Common Apps by Genres   

### 1) Google Play Store

At first, we check what "Genres" categories we have. From the output below, it's noticeable there is quite a lot of them.

In [23]:
free_ggl2.Genres.unique()

array(['Social', 'Comics', 'Tools', 'Communication', 'News & Magazines',
       'Health & Fitness', 'Education', 'Puzzle', 'Lifestyle', 'Business',
       'Shopping', 'Medical', 'Finance', 'Action', 'Personalization',
       'Casual', 'Photography', 'Travel & Local', 'Entertainment',
       'Dating', 'Productivity', 'Art & Design', 'Sports', 'Simulation',
       'Food & Drink', 'Board', 'Video Players & Editors', 'Racing',
       'Adventure', 'Books & Reference', 'House & Home',
       'Maps & Navigation', 'Events', 'Educational;Education',
       'Education;Education', 'Casino', 'Card', 'Auto & Vehicles',
       'Arcade', 'Weather', 'Beauty', 'Strategy', 'Music',
       'Libraries & Demo', 'Role Playing', 'Trivia',
       'Entertainment;Creativity', 'Parenting', 'Board;Brain Games',
       'Casual;Pretend Play', 'Educational', 'Word',
       'Action;Action & Adventure', 'Educational;Pretend Play',
       'Parenting;Education', 'Parenting;Music & Video',
       'Casual;Creativity', 'Sp

As we want to build the app in most popular categories, the long list from above can be filtered just to first 10 most occupied categories. Using groupby() function applied on DataFrame object is really powerfull tool here:

In [24]:
count_ggl = (free_ggl2.groupby('Genres').count()
             .sort_values(by='App', ascending=False)
             .rename(columns={'App':'Count'})['Count']
#              .iloc[:10]
             .to_frame()
            
            )

count_ggl['Percentage [%]'] = round((count_ggl['Count'] / count_ggl.sum()[0])*100,1)
count_ggl

,Count,Percentage [%]
Genres,,
Tools,730,8.5
Entertainment,518,6.1
Education,457,5.3
Business,400,4.7
Productivity,337,3.9
...,...,...
Card;Brain Games,1,0.0
Racing;Pretend Play,1,0.0
Card;Action & Adventure,1,0.0


For Android dataset, there is column "Category" seemingly containing similar information as the one observed above. To be sure, let's have a look on numbers:

In [25]:
count_ggl_cat = (free_ggl2.groupby('Category').count()
             .sort_values(by='App', ascending=False)
             .rename(columns={'App':'Count'})['Count']
#              .iloc[:10]
             .to_frame()
            
            )

count_ggl_cat['Percentage [%]'] = round((count_ggl_cat['Count'] / count_ggl_cat.sum()[0])*100,1)
count_ggl_cat

,Count,Percentage [%]
Category,,
FAMILY,1616,18.9
GAME,844,9.9
TOOLS,731,8.5
BUSINESS,400,4.7
PRODUCTIVITY,337,3.9
LIFESTYLE,334,3.9
FINANCE,318,3.7
MEDICAL,310,3.6
PERSONALIZATION,289,3.4


### 2) iOS App Store

The same thing applies for iOS platform:

In [26]:
free_ios2.prime_genre.unique()

array(['Social Networking', 'Photo & Video', 'Games', 'Music',
       'Reference', 'Health & Fitness', 'Weather', 'Travel', 'Shopping',
       'News', 'Navigation', 'Lifestyle', 'Entertainment', 'Food & Drink',
       'Sports', 'Finance', 'Education', 'Productivity', 'Utilities',
       'Book', 'Business', 'Catalogs', 'Medical'], dtype=object)

In [27]:
count_ios = (free_ios2.groupby('prime_genre').count()
             .sort_values(by='track_name', ascending=False)
             .rename(columns={'track_name':'Count'})['Count']
#              .iloc[:10]
             .to_frame()
            
            )

count_ios['Percentage [%]'] = round((count_ios['Count'] / count_ios.sum()[0])*100,1)
count_ios

,Count,Percentage [%]
prime_genre,,
Games,1800,59.6
Entertainment,230,7.6
Photo & Video,150,5.0
Education,115,3.8
Social Networking,96,3.2
Shopping,74,2.5
Utilities,67,2.2
Music,63,2.1
Sports,61,2.0


### 3) Conclusions

Let's start by looking at Google play frequency tables. From the first one, sorted by "Genre" column, we can see that the most of the free apps belong to "Tools" category. Second place takes entertainment. 
Table grouped by "Category" column shows slightly different distribution: "FAMILY" and "GAMES", followed by "TOOLS".

Further down the analysis I will use categories sorted by "Category" column. We can say that app such sorting is much general in comparison with sorting by "Genre", which seems way too granular and specific. 

The iOS App Store table has "Games" as the most common category. On the second place are "Entertainment".

By comparing results of both platforms we can conclude few things. In my opinion, there could some overlap of categories "FAMILY" and "GAMES" in the Google Play table, as well as "Games" and "Entertainment". In the first table for Google Play Store, I think that the "Entertainment" category could include games also.

We got the distribution of various app types on the market - it's obvious the market is dominated by apps for fun. On the other hand, fore example fitness apps seems much less popular in comparison with entertainment apps. Interesting point may be that at the Android market, "Tools" category seems much more popular than on Apple platform. This could be due the fact that Android devices are much more customizeable in comparison with Apple ones. Thus, there is probably much more apps to "tweak" your gadget for Android device.


## Most popular genres by average number of installs

In following part, we'll try to have look on average number of installs of each category. How to decide which category to pick ?
Maybe it would be convenient to look for category, that has large number of installs and relatively lower number of apps in total. This fact could mean that there is possibly small competition and the higher demand for apps from such category at the same time.

In [28]:
free_ggl2['install_num'] = (free_ggl2["Installs"]
                            .map(lambda string: float(string[:-1].replace(',','')))
                            
                           )

In [29]:
free_ggl4 = free_ggl2.groupby('Category').mean()
free_ggl4.sort_values(by='install_num', ascending = False)

,Rating,English,install_num
Category,,,
COMMUNICATION,4.121681,True,3.582147e+07
VIDEO_PLAYERS,4.025000,True,2.496488e+07
SOCIAL,4.244388,True,2.403864e+07
PHOTOGRAPHY,4.165560,True,1.802822e+07
PRODUCTIVITY,4.181752,True,1.683774e+07
GAME,4.236476,True,1.592601e+07
TRAVEL_AND_LOCAL,4.054167,True,1.437032e+07
TOOLS,4.026242,True,1.088778e+07
NEWS_AND_MAGAZINES,4.110526,True,9.926131e+06


By looking at both inspected characteristics we can see that entertaining categories are most likely saturated. If we sort the resulting dataframe by number of installs, we can see that leading positions belong to communication. 

Such categories as COMMUNICATION or SOCIAL have quite large number of installs due to outlying apps. For example, in SOCIAL category, the first place occupies - non-surprisingly- Facebook or Snapchat. Such patterns could easily skew our decisions.


In [40]:
free_ggl_inst_and_count = (free_ggl2.groupby('Category')
        .agg({'install_num':'mean', 'App':'count'})
        .sort_values(by='App', ascending=False)
        .rename(columns={'App':'app_count'})
 
)


# test['ratio'] = test['install_num'] / test['app_count'] 

free_ggl_inst_and_count.sort_values(by='install_num', ascending=False)

,install_num,app_count
Category,,
COMMUNICATION,3.582147e+07,275
VIDEO_PLAYERS,2.496488e+07,150
SOCIAL,2.403864e+07,228
PHOTOGRAPHY,1.802822e+07,254
PRODUCTIVITY,1.683774e+07,337
GAME,1.592601e+07,844
TRAVEL_AND_LOCAL,1.437032e+07,196
TOOLS,1.088778e+07,731
NEWS_AND_MAGAZINES,9.926131e+06,237


In [41]:
free_ggl2[free_ggl2['Category'] == 'SOCIAL'].sort_values(by='Installs',ascending=False).head()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver,English,install_num
4332,Facebook Lite,SOCIAL,4.3,8606259,Varies with device,"500,000,000+",Free,0,Teen,Social,"August 1, 2018",Varies with device,Varies with device,True,500000000.0
7889,Snapchat,SOCIAL,4.0,17015352,Varies with device,"500,000,000+",Free,0,Teen,Social,"July 30, 2018",Varies with device,Varies with device,True,500000000.0
7237,Qeek for Instagram - Zoom profile insta DP,SOCIAL,4.6,12111,9.1M,"500,000+",Free,0,Everyone,Social,"May 3, 2018",1.47,4.3 and up,True,500000.0
3158,Daddyhunt: Gay Dating,SOCIAL,3.9,6450,14M,"500,000+",Free,0,Mature 17+,Social,"January 22, 2018",1.3.5,4.0 and up,True,500000.0
1675,Bloglovin',SOCIAL,3.9,8936,Varies with device,"500,000+",Free,0,Everyone,Social,"May 18, 2017",Varies with device,Varies with device,True,500000.0


In [42]:
free_ggl2[free_ggl2['Category'] == 'WEATHER'].sort_values(by='Installs',ascending=False).head()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver,English,install_num
6562,New 2018 Weather App & Widget,WEATHER,4.5,2332,20M,"500,000+",Free,0,Teen,Weather,"August 3, 2018",12.2.7.3671,4.1 and up,True,500000.0
4766,GO Weather EX Theme White,WEATHER,4.2,8723,4.6M,"500,000+",Free,0,Everyone,Weather,"August 27, 2014",1.0,2.2 and up,True,500000.0
5957,MIUI Style GO Weather EX,WEATHER,4.2,2115,1.1M,"500,000+",Free,0,Everyone,Weather,"July 4, 2014",1.3,1.6 and up,True,500000.0
5866,Local Weather Forecast & Visual Widget,WEATHER,4.5,3478,6.1M,"500,000+",Free,0,Everyone,Weather,"August 2, 2018",2.9.10,4.2 and up,True,500000.0
5612,Klara weather,WEATHER,4.6,36900,4.8M,"500,000+",Free,0,Everyone,Weather,"July 28, 2018",1.3.6,4.0 and up,True,500000.0


One interesting category may be WEATHER. From the installs perspective, it lies somewhere in the middle part. But if we look at the total number of apps, it seems that there could be less apps to compete with.

App Store dataset does not have such column as installs_num, so we will approximate the popularity by number of user reviews. As we can see, using this criterion shows that weather apps have good position in the first rows of rating counts dataset.

In [50]:
free_ios2.groupby('prime_genre').mean()['rating_count_tot'].to_frame().sort_values(by='rating_count_tot', ascending=False)

,rating_count_tot
prime_genre,
Navigation,102592.000000
Reference,84258.250000
Social Networking,75253.843750
Music,55396.015873
Weather,48275.576923
Travel,34115.575758
Food & Drink,33333.923077
Photo & Video,29249.766667
Shopping,28517.729730


## Final Conclusion

Initial datasets describe apps and their statistics on both Google Play and App Store markets. We examined most common categories of apps occuring on markets. Then we took a look at popularity of app genres. Main idea was to find an app that seems popular, but the category is not saturated yet - that means there is relatively less apps than in other genres and so not too much competition. Accoriding to these observations, creating a weather app could be interesting idea.